In [31]:
# import
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
import google.generativeai as genai
from typing import IO

In [32]:
# Start Google API

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
MODEL = "gemini-1.5-flash"
genai.configure(api_key=GOOGLE_API_KEY)


In [39]:
class NoteSummarizer:
    def __init__(self):
        self.roles = {
            "concise": "You are a master summarizer. Condense the following text into clear, concise markdown bullet points, highlighting the key takeaways and actionable items. Prioritize brevity and clarity.",
            "structured": "You are a logical and organized thinker. Structure the following text into a well-formatted markdown document with headings, subheadings, and lists. Identify the main arguments, supporting evidence, and any logical connections between ideas. Use proper hierarchy and indentation.",
            "detail": "You are a meticulous analyst. Transform the following text into detailed markdown notes. Include all relevant information, even seemingly minor details. Use tables, lists, and code blocks where appropriate to present the information clearly and comprehensively.",
            "creative": "You are a creative writer with a flair for language. Convert the following text into engaging and descriptive markdown notes. Use vivid imagery, metaphors, and storytelling techniques to make the notes memorable and interesting. Focus on capturing the essence and spirit of the original text.",
            "technical": "You are a technical expert in software development. Convert the following technical documentation into well-structured markdown. Pay close attention to code examples, API specifications, and technical terminology. Use code blocks, tables, and lists to enhance readability.",
            "historian": "You are a historian specializing in capturing the essence of historical events. Convert the following text into markdown notes that emphasize the timeline, key figures, and significant events. Use timelines, maps (if applicable and you can generate them), and quotations to bring the history to life.",
            "philosopher": "You are a philosopher skilled in abstract thought. Transform the following philosophical text into well-structured markdown notes. Focus on identifying the core arguments, exploring different perspectives, and highlighting any underlying assumptions. Use quotes and examples to illustrate the concepts.",
            "teacher": "You are a teacher explaining a complex concept. Convert the following text into easy-to-understand markdown notes suitable for students. Use clear language, simple examples, and visual aids (if applicable). Break down complex ideas into smaller, digestible chunks.",
            "researcher": "You are a researcher conducting a literature review. Convert the following research papers into structured markdown notes. Focus on summarizing the key findings, methodologies, and contributions of each paper. Use citations and a consistent formatting style.",
            "assistant": "You are a highly organized personal assistant. Convert the following meeting notes into actionable markdown items. Identify tasks, deadlines, assigned individuals, and any relevant context. Use checklists, tables, and bolding to highlight important information.",
        }
        self.model = "gemini-1.5-flash"
        self.user_prompt = """
I will provide my note to you. 
If the notes knowledge point is incomplete, please provide complementary information. 
If you are not 100% sure of the answer, write「Need to collect the knowledge point」. 
If code is included in the notes, please provide the code in a code block. 
The content you provide needs to be written in markdown: Here are the notes:\n"""
        
        
    def __call__(self, file_obj: IO | None, role: str):  # Type hints and handle None
        print(role)
        if file_obj is None:
            return "No file uploaded", ""

        if role not in self.roles:
            return f"Invalid role: {role}. Available roles are: {', '.join(self.roles.keys())}", ""

        
        self.model = genai.GenerativeModel(model_name=MODEL, system_instruction=self.roles[role])

        try:
            filepath = file_obj.name
            with open(filepath, "r", encoding="utf-8") as f:
                file_content = f.read()

            messages = self.user_prompt + file_content  # Construct messages here
            response = self.model.generate_content(messages) # removed stream=stream
            return response.text

        except Exception as e:
            return f"Error: {e}", ""
        

    def fileContent(self):
        if file_obj is None:  # Handle the case where no file is uploaded
            return "No file uploaded", ""
    
        try:
            # Option 1: Read directly from the temporary file path (most efficient)
            filepath = self.file_obj.name  # Get the path of the uploaded temp file
            with open(filepath, "r", encoding="utf-8") as f: # Explicitly specify UTF-8 encoding
                file_content = f.read()
            return "File processed successfully!", file_content
    
        except Exception as e:
            return f"Error processing file: {e}", ""
    

In [40]:
import gradio as gr
    
with gr.Blocks() as app:
    file_input = gr.File(label="Upload a File")
    role = gr.Dropdown(
        ["concise", "structured", "detail", "teacher", "creative", "technical", "historian", "philosopher", "researcher", "assistant"],  # All your roles
        label="Select a role"
    )
    text_output = gr.Textbox(label="File Summary")
    process_button = gr.Button("Process File")

    markdownSummarizer = NoteSummarizer()  # Create an instance

    process_button.click(
        markdownSummarizer,
        inputs=[file_input, role],
        outputs=[text_output]
    )

app.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


concise
technical
